<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 3.2: Generators: Collections
**Prev: [Generators: Parameters](3.1_parameters.ipynb)**<br>
**Next: [Interlude: Chisel Standard Library](3.2_interlude.ipynb)**


## Motivation
Generators will frequently have to deal with variable numbers of objects, whether they are IOs, modules, or test vectors.
Collections are important building blocks for dealing with situations like this.
This module will introduce Scala collections and how to use them with Chisel generators.

## Setup

In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

Compiling /Users/dan/dev/dan-fritchman/chisel-bootcamp/Main.sc

Compiling /Users/dan/dev/dan-fritchman/chisel-bootcamp/Main.sc #2

path: String = "/Users/dan/dev/dan-fritchman/chisel-bootcamp/source/load-ivy.sc"

Note we add a new import here because `mutable.ArrayBuffer` lives in `scala.collections`.

In [2]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}
import scala.collection._

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import scala.collection._

---
# Generators and Collections<a name="generators-and-collections"></a> 
In this section, we will focus on the concept of *generators* and the use of Scala collections as a tool to implement them.
Instead of looking at Chisel code as an *instance* of a circuit, i.e. a description of a particular circuit, 
we will instead consider it to be a generator of a circuit.

We will start by considering the FIR filter from previous exercises.  

In [3]:
class My4ElementFir(b0: Int, b1: Int, b2: Int, b3: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val out = Output(UInt(8.W))
  })

  val x_n1 = RegNext(io.in, 0.U)
  val x_n2 = RegNext(x_n1, 0.U)
  val x_n3 = RegNext(x_n2, 0.U)
  io.out := io.in * b0.U(8.W) + x_n1 * b1.U(8.W) +
    x_n2 * b2.U(8.W) + x_n3 * b3.U(8.W)
}


defined class My4ElementFir

This circuit is a simple case of a generator because it can generate versions of this 4-tap filter with
different coefficients.  But what if we want the circuit to have more taps?  We are going to do this in several steps.

- Build a software *Golden Model* of a tap configurable FIR.
- Redesign our test to use this model, and confirm that it works.
- Refactor our My4ElementFir to allow an configurable number of taps.
- Test the new circuit using our new test harness.

<span style="color:blue">**Example: FIR Golden Model**</span><br><a name="fir-golden-model"></a> 
Below is a Scala software implementation of a FIR circuit.  

In [4]:
/**
  * A naive implementation of an FIR filter with an arbitrary number of taps.
  */
class ScalaFirFilter(taps: Seq[Int]) {
  var pseudoRegisters = List.fill(taps.length)(0)

  def poke(value: Int): Int = {
    pseudoRegisters = value :: pseudoRegisters.take(taps.length - 1)
    var accumulator = 0
    for(i <- taps.indices) {
      accumulator += taps(i) * pseudoRegisters(i)
    }
    accumulator
  }
}

defined class ScalaFirFilter

### Seq
Note that `taps` has become a `Seq[Int]` which means that the user of the class can pass an arbitrarily-long sequence of `Int`s when constructing the class.
### Registers
With `  var pseudoRegisters = List.fill(taps.length)(0)` we create a `List` that will hold values from previous cycles.  `List` was chosen because its syntax of adding an element to the head and removing the last element is very simple. Just about any member of the scala collections family could be used.  We are also initializing this list to contain all zeros.
### Poke
Our class adds a poke function/method that emulates putting a new input into the filter and cycling the clock.
### Updating the registers
The line `pseudoRegisters = value :: pseudoRegisters.take(taps.length - 1)` first uses the `take` method of list to keep the all but the last element of the list, then uses the `::` list concatentation operator to add `value` to the head of the reduced version of the list.
### Computing the output
A simple for loop with an accumulator sums each element of the list times its corresponding tap coefficient. The line with just `accumulator` returns that value as the function result.
## Adapting our previous test for testing our golden model
We will now use our previous work to confirm that our golden model works.  A bit of editing magic takes our previous tests harness and morphs it into...

In [5]:
val filter = new ScalaFirFilter(Seq(1, 1, 1, 1))

var out = 0

out = filter.poke(1)
println(s"out = $out")
assert(out == 1)  // 1, 0, 0, 0

out = filter.poke(4)
assert(out == 5)  // 4, 1, 0, 0
println(s"out = $out")

out = filter.poke(3)
assert(out == 8)  // 3, 4, 1, 0
println(s"out = $out")

out = filter.poke(2)
assert(out == 10)  // 2, 3, 4, 1
println(s"out = $out")

out = filter.poke(7)
assert(out == 16)  // 7, 2, 3, 4
println(s"out = $out")

out = filter.poke(0)
assert(out == 12)  // 0, 7, 2, 3
println(s"out = $out")

out = 1
out = 5
out = 8
out = 10
out = 16
out = 12


filter: ScalaFirFilter = ammonite.$sess.cmd3$Helper$ScalaFirFilter@59dee8bc
out: Int = 12

Executing the previous block demonstrates that our software model returns the same results as My4ElementFir did.


## Test circuit using the golden model.<a name="use-golden-model-as-test"></a> 
Now that we are reasonably confident about our golden model, we re-write our test to compare the circuit outputs with the output of the golden model, instead of using laboriously hand-crafted examples.
What follows is a quick first pass to do it.

In [6]:
val goldenModel = new ScalaFirFilter(Seq(1, 1, 1, 1))

Driver(() => new My4ElementFir(1, 1, 1, 1)) {
  c => new PeekPokeTester(c) {
    for(i <- 0 until 100) {
      val input = scala.util.Random.nextInt(8)

      val goldenModelResult = goldenModel.poke(input)

      poke(c.io.in, input)

      expect(c.io.out, goldenModelResult, s"i $i, input $input, gm $goldenModelResult, ${peek(c.io.out)}")

      step(1)
    }
  }
}

[info] [0.002] Elaborating design...
[info] [1.071] Done elaborating.
Total FIRRTL Compile Time: 703.1 ms
Total FIRRTL Compile Time: 67.8 ms
End of dependency graph
Circuit state created
[info] [0.003] SEED 1575031539616
test My4ElementFir Success: 100 tests passed in 105 cycles taking 0.126268 seconds
[info] [0.111] RAN 100 CYCLES PASSED


goldenModel: ScalaFirFilter = ammonite.$sess.cmd3$Helper$ScalaFirFilter@4f44dbfa
res5_1: Boolean = true

Our test runs for 100 cycles, and checks that the two different methods, hardware and software, are in sync at each step.

### Things to watch out for
(i.e., mistakes we actually committed while writing this.)

1. Getting the step in the right place. Software and hardware execute differently; it's easy to get this wrong.
1. This test is weak because it is very sensitive to how the IOs and registers are sized. Implementing a software golden model that observes wrapping behavior at arbitrary data bit widths can be complicated.  Here we just make sure that we only pass in values that fit.

<span style="color:blue">**Example: Parameterized FIR Generator**</span><br><a name="fir-golden-model"></a> 
Below we have created a new Filter class, `MyManyElementsFilter` that takes a `Seq` of constants to use for taps.  This list can be any number of elements.
For good measure a `bitWidth` has been added that allows us to control the sizes of numbers that can be handled by our circuit.
In response the variable length we have had to refactor the creation of registers and how they are connected.
The methodology used below uses a simple subset of the available library of collection functions.
Later sections show how to more succinctly express the behavior in a way that also makes what is happening clearer.

In [8]:
class MyManyElementFir(consts: Seq[Int], bitWidth: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(bitWidth.W))
    val out = Output(UInt(bitWidth.W))
  })

  val regs = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until consts.length) {
      if(i == 0) regs += io.in
      else       regs += RegNext(regs(i - 1), 0.U)
  }
  
  val muls = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until consts.length) {
      muls += regs(i) * consts(i).U
  }

  val scan = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until consts.length) {
      if(i == 0) scan += muls(i)
      else scan += muls(i) + scan(i - 1)
  }

  io.out := scan.last
}

defined class MyManyElementFir

#### How we did it
There are three parallel sections starting at lines 7, 13, and 18.
We are using a Scala collection type called `ArrayBuffer`.
`ArrayBuffer` allows you to append elements using the `+=` operator (also insert and delete, but we don't need this).
First, we create an ArrayBuffer `regs` whose elements will be `UInt`s.
Then iterate over the taps, adding the input as the first element followed by creating registers using RegNext which connect the input of the register to the previous element (`regs(i-1)`) and initializes it to unsigned zero (`0.U`).
These registers will hold the previous values of inputs as they are needed.

Next, we create another ArrayBuffer `muls` of `UInt`s.
Each element of muls will be a node whose i-th element is the product of the `regs(i)` and `const(i)`.

Note the use of the `scan.last` method.
It takes the last element of a collection, and is a more elegant alternative to `regs(i - 1)` used during the `regs` construction.

### Does it behave the same as `My4ElementFir`?
A good first test of our new version is to see if it can pass the test we just applied to the
`My4ElementFir`.
We create an instance of `MyManyElementFir` and run even more data through it.

In [9]:
val goldenModel = new ScalaFirFilter(Seq(1, 1, 1, 1))

Driver(() => new MyManyElementFir(Seq(1, 1, 1, 1), 8)) {
  c => new PeekPokeTester(c) {
    for(i <- 0 until 100) {
      val input = scala.util.Random.nextInt(8)

      val goldenModelResult = goldenModel.poke(input)

      poke(c.io.in, input)

      expect(c.io.out, goldenModelResult, s"i $i, input $input, gm $goldenModelResult, ${peek(c.io.out)}")

      step(1)
    }
  }
}

[info] [0.000] Elaborating design...
[info] [0.070] Done elaborating.
Total FIRRTL Compile Time: 74.1 ms
Total FIRRTL Compile Time: 58.3 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1575031995809
test MyManyElementFir Success: 100 tests passed in 105 cycles taking 0.060921 seconds
[info] [0.060] RAN 100 CYCLES PASSED


goldenModel: ScalaFirFilter = ammonite.$sess.cmd3$Helper$ScalaFirFilter@3ab4a68
res8_1: Boolean = true

### Now let's test a bunch of different sized FIR filters
We create some helper functions: `r` which gets a random number; `runOneTest` which creates a golden model and a hardware simulation of a filter for a particular set of taps, and then runs at least twice the number of taps worth of data through the filter.

In [10]:
/** a convenience method to get a random integer
  */
def r(): Int = {
  scala.util.Random.nextInt(1024)
}

/**
  * run a test comparing software and hardware filters
  * run for at least twice as many samples as taps
  */
def runOneTest(taps: Seq[Int]) {
  val goldenModel = new ScalaFirFilter(taps)

  Driver(() => new MyManyElementFir(taps, 32)) {
    c => new PeekPokeTester(c) {
      for(i <- 0 until 2 * taps.length) {
        val input = r()

        val goldenModelResult = goldenModel.poke(input)

        poke(c.io.in, input)

        expect(c.io.out, goldenModelResult, s"i $i, input $input, gm $goldenModelResult, ${peek(c.io.out)}")

        step(1)
      }
    }
  }
}

for(tapSize <- 2 until 100 by 10) {
  val taps = Seq.fill(tapSize)(r())  // create a sequence of random coefficients

  runOneTest(taps)
}

[info] [0.000] Elaborating design...
[info] [0.025] Done elaborating.
Total FIRRTL Compile Time: 64.8 ms
Total FIRRTL Compile Time: 96.1 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1575032037992
test MyManyElementFir Success: 4 tests passed in 9 cycles taking 0.003419 seconds
[info] [0.004] RAN 4 CYCLES PASSED
[info] [0.000] Elaborating design...
[info] [0.104] Done elaborating.
Total FIRRTL Compile Time: 99.7 ms
Total FIRRTL Compile Time: 100.7 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1575032038279
test MyManyElementFir Success: 24 tests passed in 29 cycles taking 0.089063 seconds
[info] [0.064] RAN 24 CYCLES PASSED
[info] [0.000] Elaborating design...
[info] [0.011] Done elaborating.
Total FIRRTL Compile Time: 103.0 ms
Total FIRRTL Compile Time: 145.0 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1575032038927
test MyManyElementFir Success: 44 tests passed in 49 cycles taking 0.186091 seconds
[info] [0.181] R

defined function r
defined function runOneTest

### Just for fun, let's make a bigger one
The following will run a single test on a 500 tap
FIR filter.  It can take a minute or so to run.
(Hint: Watch for the Scala ● to change to Scala ○ on the Toolbar when the execution completes.)

In [11]:
runOneTest(Seq.fill(500)(r()))

[info] [0.000] Elaborating design...
[info] [0.086] Done elaborating.
Total FIRRTL Compile Time: 689.4 ms
Total FIRRTL Compile Time: 588.6 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1575032048695
test MyManyElementFir Success: 1000 tests passed in 1005 cycles taking 7.489493 seconds
[info] [7.445] RAN 1000 CYCLES PASSED


In [12]:
val taps = Seq.fill(500)(r())

val goldenModel = new ScalaFirFilter(taps)

Driver(() => new MyManyElementFir(taps, 32)) {
  c => new PeekPokeTester(c) {
    for(i <- 0 until 100) {
      val input = r()

      val goldenModelResult = goldenModel.poke(input)

      poke(c.io.in, input)

      expect(c.io.out, goldenModelResult, s"i $i, input $input, gm $goldenModelResult, ${peek(c.io.out)}")

      step(1)
    }
  }
}

[info] [0.000] Elaborating design...
[info] [0.087] Done elaborating.
Total FIRRTL Compile Time: 793.8 ms
Total FIRRTL Compile Time: 665.4 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1575032061945
test MyManyElementFir Success: 100 tests passed in 105 cycles taking 0.982896 seconds
[info] [0.876] RAN 100 CYCLES PASSED


taps: Seq[Int] = List(
  1003,
  161,
  264,
  100,
  656,
  789,
  493,
  924,
  111,
  759,
  819,
  583,
  931,
  638,
  56,
  839,
  742,
  450,
  866,
  44,
  178,
  665,
  82,
  603,
  478,
  826,
  263,
  200,
  251,
  722,
  588,
  299,
  284,
  634,
  766,
  835,
  168,
  122,
...
goldenModel: ScalaFirFilter = ammonite.$sess.cmd3$Helper$ScalaFirFilter@47c238b8
res11_2: Boolean = true

---
# Hardware Collections

<span style="color:blue">**Example: Add run-time configurable taps to our FIR**</span><br>
The following code adds an additional `consts` vector to the IO of our FIR generator which allows the coefficients to be changed externally after circuit generation.
This is done with the Chisel collection type `Vec`.
`Vec` supports many of the scala collection methods but it can only contain Chisel hardware elements.
`Vec` should only be used in situations where ordinary Scala collections won't work.  
Basically this is in one of two situations.
1. You need a collection of elements in a Bundle, typically a Bundle that will be used as IO.
1. You need to access the collection via an index that is part of the hardware (think Register File).


In [13]:
class MyManyDynamicElementVecFir(length: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val out = Output(UInt(8.W))
    val consts = Input(Vec(length, UInt(8.W)))
  })

  // Reference solution
  val regs = RegInit(VecInit(Seq.fill(length - 1)(0.U(8.W))))
  for(i <- 0 until length - 1) {
      if(i == 0) regs(i) := io.in
      else       regs(i) := regs(i - 1)
  }
  
  val muls = Wire(Vec(length, UInt(8.W)))
  for(i <- 0 until length) {
      if(i == 0) muls(i) := io.in * io.consts(i)
      else       muls(i) := regs(i - 1) * io.consts(i)
  }

  val scan = Wire(Vec(length, UInt(8.W)))
  for(i <- 0 until length) {
      if(i == 0) scan(i) := muls(i)
      else scan(i) := muls(i) + scan(i - 1)
  }

  io.out := scan(length - 1)
}

defined class MyManyDynamicElementVecFir

In [14]:
val goldenModel = new ScalaFirFilter(Seq(1, 1, 1, 1))

Driver(() => new MyManyDynamicElementVecFir(4)) {
  c => new PeekPokeTester(c) {
    poke(c.io.consts(0), 1)
    poke(c.io.consts(1), 1)
    poke(c.io.consts(2), 1)
    poke(c.io.consts(3), 1)
    for(i <- 0 until 100) {
      val input = scala.util.Random.nextInt(8)

      val goldenModelResult = goldenModel.poke(input)

      poke(c.io.in, input)

      expect(c.io.out, goldenModelResult, s"i $i, input $input, gm $goldenModelResult, ${peek(c.io.out)}")

      step(1)
    }
  }
}

[info] [0.000] Elaborating design...
[info] [0.098] Done elaborating.
Total FIRRTL Compile Time: 57.7 ms
Total FIRRTL Compile Time: 30.9 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1575032281633
test MyManyDynamicElementVecFir Success: 100 tests passed in 105 cycles taking 0.056403 seconds
[info] [0.045] RAN 100 CYCLES PASSED


goldenModel: ScalaFirFilter = ammonite.$sess.cmd3$Helper$ScalaFirFilter@3f24dc68
res13_1: Boolean = true

<span style="color:red">**Exercise: 32-bit RISC-V Processor**</span><br><a name="fir-golden-model"></a>

A [register file](https://en.wikipedia.org/wiki/Register_file) is an important building block for making a processor.
A register file is an array of registers that can be read from or written to via a number of read or write ports.
Each port consists of an address and data field.

The [RISC-V instruction set architecture](https://riscv.org/specifications/) defines several variants, the simplest of which is called RV32I.
RV32I has a size-32 array of 32-bit registers.
**The register at index 0 (the first register) is always zero when you read from it, regardless of what you write to it** (it's often useful to have 0 handy).

Implement a register file for RV32I with a single write port and a paramterized number of read ports.
Writes will only be performed when `wen` (write enable) is asserted.

In [23]:
class RegisterFile(readPorts: Int) extends Module {
    require(readPorts >= 0)
    val io = IO(new Bundle {
        val wen   = Input(Bool())
        val waddr = Input(UInt(5.W))
        val wdata = Input(UInt(32.W))
        val raddr = Input(Vec(readPorts, UInt(5.W)))
        val rdata = Output(Vec(readPorts, UInt(32.W)))
    })
    
    // A Register of a vector of UInts
    val reg = RegInit(VecInit(Seq.fill(32)(0.U(32.W))))
    
    when(io.wen) {
        reg(io.waddr) := io.wdata
    }
    for(i <- 0 until readPorts) {
      when(io.raddr(i) === 0.U) { io.rdata(i) := 0.U }
      .otherwise{io.rdata(i) := reg(i)}
    }

}

defined class RegisterFile

In [24]:
chisel3.iotesters.Driver(() => new RegisterFile(2) ) { c => new PeekPokeTester(c) {
    def readExpect(addr: Int, value: Int, port: Int = 0): Unit = {
        poke(c.io.raddr(port), addr)
        expect(c.io.rdata(port), value)
    }
    def write(addr: Int, value: Int): Unit = {
        poke(c.io.wen, 1)
        poke(c.io.wdata, value)
        poke(c.io.waddr, addr)
        step(1)
        poke(c.io.wen, 0)
    }
    // everything should be 0 on init
    for (i <- 0 until 32) {
        readExpect(i, 0, port = 0)
        readExpect(i, 0, port = 1)
    }

    // write 5 * addr + 3
    for (i <- 0 until 32) {
        write(i, 5 * i + 3)
    }

    // check that the writes worked
    for (i <- 0 until 32) {
        readExpect(i, if (i == 0) 0 else 5 * i + 3, port = i % 2)
    }

}}

[info] [0.000] Elaborating design...
[info] [0.067] Done elaborating.
Total FIRRTL Compile Time: 82.4 ms
Total FIRRTL Compile Time: 54.1 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1575033033858
[info] [0.057] EXPECT AT 32   io_rdata_0 got 3 expected 13 FAIL
[info] [0.058] EXPECT AT 32   io_rdata_1 got 8 expected 18 FAIL
[info] [0.058] EXPECT AT 32   io_rdata_0 got 3 expected 23 FAIL
[info] [0.066] EXPECT AT 32   io_rdata_1 got 8 expected 28 FAIL
[info] [0.067] EXPECT AT 32   io_rdata_0 got 3 expected 33 FAIL
[info] [0.069] EXPECT AT 32   io_rdata_1 got 8 expected 38 FAIL
[info] [0.070] EXPECT AT 32   io_rdata_0 got 3 expected 43 FAIL
[info] [0.072] EXPECT AT 32   io_rdata_1 got 8 expected 48 FAIL
[info] [0.073] EXPECT AT 32   io_rdata_0 got 3 expected 53 FAIL
[info] [0.075] EXPECT AT 32   io_rdata_1 got 8 expected 58 FAIL
[info] [0.075] EXPECT AT 32   io_rdata_0 got 3 expected 63 FAIL
[info] [0.076] EXPECT AT 32   io_rdata_1 got 8 expected 68 FAIL
[info] [0.07

res23: Boolean = false

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
    when (io.wen) {
        reg(io.waddr) := io.wdata
    }
    for (i &lt;- 0 until readPorts) {
        when (io.raddr(i) === 0.U) {
            io.rdata(i) := 0.U
        } .otherwise {
            io.rdata(i) := reg(io.raddr(i))
        }
    }

</pre></article></div></section></div>

---
# You're done!

[Return to the top.](#top)